In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.text import *

## prepare data 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = Path('./drive/MyDrive/hUlMona')
path.ls()

[PosixPath('drive/MyDrive/hUlMona/data'),
 PosixPath('drive/MyDrive/hUlMona/Ar-LM_unfreeze_1e-4_after3_2_acc475.pth'),
 PosixPath('drive/MyDrive/hUlMona/models'),
 PosixPath('drive/MyDrive/hUlMona/.ipynb_checkpoints')]

In [5]:
import string

#preparing punctuations list
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ«»'''
english_punctuations = string.punctuation
punc_to_remove = ''.join(set(arabic_punctuations + english_punctuations))
punc_to_keep = '+'
punc_to_escape = '''[]-^'''
for p in punc_to_keep: punc_to_remove = punc_to_remove.replace(p, '')
for p in punc_to_escape: punc_to_remove = punc_to_remove.replace(p, '\\{}'.format(p))
print(punc_to_remove)

def pre_process(text):
    text = text.replace('\\n', ' ').replace('\n', ' ')
    text = text.replace('؛', '،')
    text = re.sub(r'\([^)]+\)', '', text)  # remove parentheses and everything in between
    text = re.sub(r'[a-zA-Z]', '', text)  # remove non-arabic characters
    text = re.sub(r'\d+(\.\d+)?', ' رقم ', text)  # replace numbers by special token
    for p in punc_to_remove: text = text.replace(p, '')  # remove punctuations
    text = re.sub(r'(.)\1{2,}', r'\1', text)  # remove repeated chars
    text = re.sub(r'\s+', r' ', text)
    return text

pretrained_fnames=['Ar-LM_unfreeze_1e-4_after3_2_acc475','itos']

from sklearn.model_selection import train_test_split

df = pd.read_csv(path/'data'/'RES2.csv')
column_names = ["polarity", "text"]
df = df.reindex(columns=column_names)

for i,text in enumerate(df["text"]):
    df["text"][i] = pre_process(text)


print(df.shape)
np.random.seed(42)
df_train, df_val = train_test_split(df, test_size=0.2)
print(df_train.shape, df_val.shape)
#df.columns
df_train.head()

«:'.“`>”(~%)¦@,!;?}»…<\]=_*\ـ&،×÷#$؛؟\-\^/"|{–\[


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(2642, 2)
(2113, 2) (529, 2)


,polarity,text
1149,1,مطعم ممتاز لولا بطء الخدمة وارتفاع أسعار الوجب...
965,-1,لا أنصح به سيء والأكل بارد والسعر مرتفع النظاف...
2398,0,مطعم تركي طعام جيد و لذيذ لكنه بالنسبة للطفل ح...
440,1,ابراهيم المطعم من تصميمه و نظافته مبين ان جيد ...
508,1,الطعام الأمريكي مكان يصلح لمحبي الأستيك الأمري...


In [6]:
data_clas = load_data(path/'data/', 'data_clas_test3.pkl', bs=32)

/usr/local/lib/python3.7/dist-packages/fastai/text/data.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sort_idx = np.concatenate(np.random.permutation(ck_idx[1:])) if len(ck_idx) > 1 else np.array([],dtype=np.int)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [7]:
data_clas.show_batch()

/usr/local/lib/python3.7/dist-packages/fastai/text/data.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sort_idx = np.concatenate(np.random.permutation(ck_idx[1:])) if len(ck_idx) > 1 else np.array([],dtype=np.int)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


text,target
تجربة مطعم xxunk لم تكن xxunk ليس xxunk على المطعم و لكن xxunk xxunk هنا xxunk لما xxunk في تجربتي الاولى للمطعم الذي يظهر بأنه مطعم فاخر تم حجز xxunk عبر xxunk لي و قد xxunk على xxunk xxunk الشخص xxunk و xxunk حيث للاسف ان التعامل هناك حسب هذه xxunk عند الوصول للافطار في رمضان قبل xxunk xxunk اي xxunk xxunk انه لم يتم اي حجز و عند السؤال,-1
هذا أفضل بكثير لتناول الشاي بعد xxunk إنه xxunk xxunk بالفعل إنه xxunk xxunk وهو يستحق ذلك لقد جربنا تناول الشاي بعد xxunk يوم السبت xxunk xxunk xxunk xxunk كان xxunk xxunk ورغم ذلك فقد كان ما كان xxunk xxunk عندما طلبنا xxunk xxunk سمك السلمون xxunk xxunk الطعام xxunk آخر xxunk لنا إن كل شيء معد xxunk ولكن ذلك لم يكن xxunk لم تحصل xxunk إلا عندما xxunk بأن,-1
اكل xxunk خدمة xxunk انا كنت بحب المحل ده xxunk ما روحت xxunk في فرع سيتي xxunk اسوأ مرة اكلت فيها طعم xxunk و اكل بارد اما الخدمة xxunk اسوأ خدمة xxunk في حياتي اي حد xxunk منه حاجة xxunk xxunk و يروح xxunk ربع ساعة كل ده كوم xxunk هناك كوم تاني الوحيد اللي xxunk xxunk مختلف كان xxunk اسود في اسود للاسف xxunk اسمه ده من اول اليوم,-1
هذا المكان لا xxunk الزوار xxunk xxunk هم افراد من دون xxunk لقد xxunk جدا معاملة الشخص xxunk على باب xxunk xxunk حيث xxunk بأنه مدير المكان وكان xxunk xxunk xxunk xxunk حيث كنا زوار xxunk غير xxunk و xxunk نزور الغردقة xxunk مرة و طلبنا منه قطع xxunk دخول حيث كنا xxunk بالعشاء و تناول السوشي و xxunk xxunk و من ثم xxunk xxunk إن كنا xxunk أم xxunk,-1
اسوء xxunk علي وجه xxunk xxunk xxunk بي xxunk اني ذهبت الي الكثير من xxunk و المقهي في حياتي و لم xxunk اسوء من هاذا الكافيه حيثو المعاملة سيئه الي xxunk الحدود و لا xxunk xxunk نصف المشروبات xxunk في xxunk و المقاهي xxunk xxunk انهم xxunk قصم الغير xxunk xxunk xxunk xxunk طاولة واحدة لا فرق xxunk و بين قصم xxunk شئ الي xxunk xxunk التدخين ولا حتي xxunk,-1


## Classifier

In [11]:
config = awd_lstm_lm_config.copy()
config['n_hid'] = 1150
del config['tie_weights']
del config['out_bias']

learn_clas = text_classifier_learner(data_clas, AWD_LSTM, config=config, drop_mult=0.2, metrics=[accuracy,FBeta(average='weighted')])
# learn_clas.load('test3_first_acc685_f643') #0.820416
learn_clas.load('test3_second_acc690_f665_min4') #0.839319

RNNLearner(data=TextClasDataBunch;

Train: LabelList (2113 items)
x: TextList
مطعم ممتاز xxunk بطء الخدمة xxunk أسعار الوجبات من مطاعم الوجبات السريعة المميزة بالنظافة وجودة الأكل يعاب عليه بطء الخدمة xxunk الأسعار المستمر,لا أنصح به سيء والأكل بارد والسعر مرتفع النظافه غير جيدة xxunk مقارنه xxunk خارج السعوديه ايضاً العماله تعاملهم غير جيد xxunk المطلوب,مطعم تركي طعام جيد و لذيذ لكنه بالنسبة xxunk حار يمكن ان تطلب xxunk أخرى xxunk الدجاج للأطفال على xxunk حال لكن معظم xxunk ذات طعم حار و الشيء اللذيذ جداً هو الخبز الطازج الذي xxunk على xxunk لذيذ جداً لدرجة أنني xxunk معي قطعة خبز من المطعم و لكن هناك تأخر في تلبية الطلب نظرا xxunk لديه مكان للجلوس خارج المطعم xxunk و أيضاً في الداخل,ابراهيم المطعم من xxunk و xxunk xxunk ان جيد جدا و الاسعار طبعا فيه مرتفعه نظرا للمأكولات الجيده xxunk xxunk ان يكون xxunk بالاسعار و جوده في الماكولات,الطعام xxunk مكان يصلح لمحبي xxunk xxunk حيث يتم xxunk xxunk جيدة جدا والسعر مناسب والمكان مناسب للشباب
y: CategoryList
1,-1,0,1,1
Path: drive/MyDrive/hUl

In [10]:
print(df_val.head())
print(learn_clas.predict("ممتاز+أكل+لبناني+بقائمة+متنوعة+وطعام+لذيذ+وديك"))

      polarity                                               text
1598         1  الاكل المجري التقليدي هالمطعم كان من ضمن جولة ...
2308         1  شكرا بجد المطعم جميل جدا والافراد الموجود هايل...
2386         1  ممتاز أكل لبناني بقائمة متنوعة وطعام لذيذ وديك...
978          1  أكل هندي مكان راقي مطعم جميل و فخم و الأكل لذي...
368          1  تجربة لطيفة مطعم لطيف وتحربة شيقة وفنجان من ال...
(Category tensor(2), tensor(2), tensor([0.0377, 0.0077, 0.9546]))


### Results
|Dataset|epoch|train_loss|valid_loss|accuracy|f_beta  |seed|test split|
|-------|-----|----------|----------|--------|--------|----|----------|
|ASTD-B |6	  |0.438674  |0.448438  |0.864662|0.857988|42  |0.1       |
|ASTD   |6	  |0.659341  |0.857126  |0.698801|0.677395|42  |0.2       |
|AJGT   |5    |0.429798	 |0.452757	|0.805556|0.745665|14  |0.2       |
|LEV    |16   |1.185471  |1.251261  |0.523750|0.511322|14  |0.2       |
|HARD   |7    |0.121737  |0.128373  |0.956695|0.956682|43  |0.2       |